# Backtesting Simple Strategies

## Importing Libraries

In [1]:
import pandas as pd
import yfinance as yf
import backtrader as bt
from datetime import datetime
import matplotlib.pyplot as plt

## Data Acquisition

In [2]:
data = yf.download('GC=F', start='2020-01-01', end='2024-01-01')

[*********************100%***********************]  1 of 1 completed


In [3]:
data.to_csv('GOLD_data.csv')

In [4]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,1518.099976,1528.699951,1518.000000,1524.500000,1524.500000,214
2020-01-03,1530.099976,1552.699951,1530.099976,1549.199951,1549.199951,107
2020-01-06,1580.000000,1580.000000,1560.400024,1566.199951,1566.199951,416
2020-01-07,1558.300049,1576.300049,1558.300049,1571.800049,1571.800049,47
2020-01-08,1579.699951,1604.199951,1552.300049,1557.400024,1557.400024,236


In [5]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000,1007.000000
mean,1828.224827,1837.602384,1819.079146,1828.325718,1828.325718,4385.268123
std,114.159337,113.764866,114.653137,114.046079,114.046079,24057.326026
min,1469.300049,1484.000000,1452.099976,1477.300049,1477.300049,0.000000
25%,1759.050049,1767.200012,1749.900024,1757.799988,1757.799988,83.000000
50%,1826.099976,1833.900024,1817.500000,1828.699951,1828.699951,248.000000
75%,1920.049988,1927.550049,1911.549988,1919.099976,1919.099976,666.000000
max,2081.600098,2130.199951,2066.500000,2081.899902,2081.899902,251274.000000


### Data Cleaning

In [6]:
data.isna().any()

Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

In [7]:
data.duplicated().any()

np.False_

### SMA Strategy Definition

In [8]:
class SMAStrategy(bt.SignalStrategy):
    def __init__(self):

        #Define moving averages
        self.sma30 = bt.indicators.SimpleMovingAverage(self.data.close, period=30)
        self.sma100 = bt.indicators.SimpleMovingAverage(self.data.close, period=100)

        #Add buy/sell signals
        self.signal_add(bt.SIGNAL_LONG, self.data.close > self.sma30)
        self.signal_add(bt.SIGNAL_SHORT, self.data.close < self.sma30)

        #Mantain signals
        self.addsignal(bt.SIGNAL_LONG, self.data.close > self.sma30)
        self.addsignal(bt.SIGNAL_SHORT, self.data.close < self.sma30)


### Backtesting Configuration & Execution

In [9]:
# Define una estrategia básica
class SmaCross(bt.Strategy):
    def __init__(self):
        self.sma1 = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
        self.sma2 = bt.indicators.SimpleMovingAverage(self.data.close, period=30)

    def next(self):
        if self.sma1 > self.sma2:
            self.buy()
        elif self.sma1 < self.sma2:
            self.sell()

# Crea una instancia de Cerebro
cerebro = bt.Cerebro()

# Carga los datos desde un archivo CSV usando GenericCSVData
datafeed = bt.feeds.GenericCSVData(
    dataname='GOLD_data.csv',
    fromdate=datetime(2020, 8, 11),
    todate=datetime(2023, 12, 15),
    nullvalue=0.0,
    dtformat='%Y-%m-%d',
    datetime=0,
    open=1,
    high=2,
    low=3,
    close=4,
    volume=5,
    openinterest=-1  # Si tu archivo no tiene open interest, usa -1
)

cerebro.adddata(datafeed)

# Establece el capital inicial
cerebro.broker.set_cash(100000)

# Añade la estrategia al cerebro
cerebro.addstrategy(SmaCross)

# Ejecuta el backtesting
cerebro.run()

# Grafica los resultados
cerebro.plot()

<IPython.core.display.Javascript object>

In [10]:
cerebro = bt.Cerebro()

#Add strategy to cerebro
cerebro.addstrategy(SMAStrategy)

#Upload data
datafeed = bt.feeds.YahooFinance(dataname='GOLD_data.csv', fromdate=pd.Timestamp('2020-08-11'), todate=pd.Timestamp('2023-12-15'))
cerebro.adddata(datafeed)

#Set initial capital
cerebro.broker.set_cash(100000)

#Set order size
cerebro.addsizer(bt.sizers.FixedSize, stake=10)

#Execute backtesting
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

/home/raulserrano/DATA_SCIENCE/proyectos/Backtesting-1-try/.venv/lib/python3.12/site-packages/backtrader/cerebro.py:670: SyntaxWarning: invalid escape sequence '\*'
  '''Adds a callback to get messages which would be handled by the
/home/raulserrano/DATA_SCIENCE/proyectos/Backtesting-1-try/.venv/lib/python3.12/site-packages/backtrader/cerebro.py:712: SyntaxWarning: invalid escape sequence '\*'
  '''Adds a callback to get messages which would be handled by the


AttributeError: 'YahooFinance' object has no attribute 'setenvironment'

### Ploting Backtesting Results

In [ ]:
# Graficar los resultados
cerebro.plot()

NameError: name 'cerebro' is not defined